In [ ]:
input_sentence="diffusion works better than autoregression"

# building vocabulary
vocab=sorted(list(set(input_sentence)))

vocab_size=len(vocab)
char_to_indx={char:i for i,char in enumerate(vocab)}
indx_to_char={i:char for i,char in enumerate(vocab)}

print(vocab_size)
print(char_to_indx)
print(indx_to_char)

17
{' ': 0, 'a': 1, 'b': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'k': 9, 'n': 10, 'o': 11, 'r': 12, 's': 13, 't': 14, 'u': 15, 'w': 16}
{0: ' ', 1: 'a', 2: 'b', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'k', 10: 'n', 11: 'o', 12: 'r', 13: 's', 14: 't', 15: 'u', 16: 'w'}


In [ ]:
# converting text into numerical format

text_to_numeric=list(char_to_indx[ch] for ch in input_sentence)
print(text_to_numeric)


[3, 8, 5, 5, 15, 13, 8, 11, 10, 0, 16, 11, 12, 9, 13, 0, 2, 4, 14, 14, 4, 12, 0, 14, 7, 1, 10, 0, 1, 15, 14, 11, 12, 4, 6, 12, 4, 13, 13, 8, 11, 10]


In [ ]:
# creating input-target pair (core step of auto regression)
x_indx=text_to_numeric[:-1]
y_indx=text_to_numeric[1:]

print(x_indx)
print(y_indx)

[3, 8, 5, 5, 15, 13, 8, 11, 10, 0, 16, 11, 12, 9, 13, 0, 2, 4, 14, 14, 4, 12, 0, 14, 7, 1, 10, 0, 1, 15, 14, 11, 12, 4, 6, 12, 4, 13, 13, 8, 11]
[8, 5, 5, 15, 13, 8, 11, 10, 0, 16, 11, 12, 9, 13, 0, 2, 4, 14, 14, 4, 12, 0, 14, 7, 1, 10, 0, 1, 15, 14, 11, 12, 4, 6, 12, 4, 13, 13, 8, 11, 10]


In [ ]:
# encode inputs
import numpy as np
np.random.seed(42)

embedding_dimention=9 # D
E=np.random.randn(vocab_size, embedding_dimention) # will use this embedding lookup at run time
print(E)




[[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
   1.57921282  0.76743473 -0.46947439]
 [ 0.54256004 -0.46341769 -0.46572975  0.24196227 -1.91328024 -1.72491783
  -0.56228753 -1.01283112  0.31424733]
 [-0.90802408 -1.4123037   1.46564877 -0.2257763   0.0675282  -1.42474819
  -0.54438272  0.11092259 -1.15099358]
 [ 0.37569802 -0.60063869 -0.29169375 -0.60170661  1.85227818 -0.01349722
  -1.05771093  0.82254491 -1.22084365]
 [ 0.2088636  -1.95967012 -1.32818605  0.19686124  0.73846658  0.17136828
  -0.11564828 -0.3011037  -1.47852199]
 [-0.71984421 -0.46063877  1.05712223  0.34361829 -1.76304016  0.32408397
  -0.38508228 -0.676922    0.61167629]
 [ 1.03099952  0.93128012 -0.83921752 -0.30921238  0.33126343  0.97554513
  -0.47917424 -0.18565898 -1.10633497]
 [-1.19620662  0.81252582  1.35624003 -0.07201012  1.0035329   0.36163603
  -0.64511975  0.36139561  1.53803657]
 [-0.03582604  1.56464366 -2.6197451   0.8219025   0.08704707 -0.29900735
   0.09176078 -1.98756

In [ ]:
# embedding look up at a time

# for x
# for t in range(len(x_indx)):
#   x_t=E[x_indx[t]]
#   print(x_t.shape) # (9,)

# for y
# for t in range(len(y_indx)):
#   y_true_t=y_indx[t]
#   print(y_true_t)


In [ ]:
# some things to remeber while working with RNN
# vocab size = V
# embedding size = D (number of dimention)
# hidden size = H
# sequence length = T (how many time steps)

# so in RNN with embeddings
# input size = embedding dimention
# hidden size = H
# output size = vocab size

# V=vocab_size # 17
D=embedding_dimention # 9
H=64

# parameters initialization
# W: hidden to hidden weights
# U: input to hidden weights
# b: hidden bias
# V: hidden to output weights
# C: output bias

W=np.random.randn(H,H)*0.01 # (64,64)
U=np.random.randn(H,D)*0.01 # (64,9)
V=np.random.randn(vocab_size,H)*0.01 # (17,64)

b=np.zeros((H,1)) # (64,1)
c=np.zeros((vocab_size,1)) # (17,1)


In [ ]:
# for t in range(len(x_indx)):
#   x_t=E[x_indx[t]]
#   x_t=x_t.reshape(-1,1)
#   print(x_t.shape) # (9,1)

In [ ]:
# activation function

def tanh(s):
    return np.tanh(s)

def derivative_tanh(s):
    return 1-np.tanh(s)**2

def softmax(o):
    e = np.exp(o - np.max(o))
    return e / np.sum(e, axis=0, keepdims=True)

In [ ]:
# loss function
# cross entropy loss function

def cross_entropy_loss(y_pred, y_true):
    eps=1e-15
    loss=0.0
    for t in range(len(y_true)):
        # loss-=np.sum(y_true[t]*np.log(y_pred[t]+eps))
        loss-=float(np.log(y_pred[t][y_true[t]]+eps))
    return loss


In [ ]:
# forward pass

def forward_pass(W,U,V,b,c,a_prev,x_indx,E):
  s,a,o,y_pred={},{},{},{}
  a[-1]=a_prev # (64,1)

  for t in range(len(x_indx)):
      x_t=E[x_indx[t]]
      x_t=x_t.reshape(-1,1) # (9,1)

      s[t]=W@a[t-1]+U@x_t+b # (64,1)
      a[t]=tanh(s[t]) # (64,1)
      o[t]=V@a[t]+c # (17,1)
      y_pred[t]=softmax(o[t])

  return s,a,o,y_pred


In [ ]:
# a_prev=np.zeros((H,1)) # (64,1)
# s,a,o,y_pred=forward_pass(W,U,V,b,c,a_prev,x_indx,E)


In [ ]:
# print(y_pred[0].shape)
# print(a[0].shape)
# print(U.shape)

In [ ]:
# y_true=np.zeros((vocab_size, 1))
# for t in range(len(y_indx)):
#   y_true[y_indx[t]]=1


In [ ]:
# back propagation through time

def BPTT(y_pred,U,V,W,b,c,E,x_indx,a,s,y_indx):

  dL_dV=np.zeros_like(V)
  dL_dU=np.zeros_like(U)
  dL_dW=np.zeros_like(W)
  dL_db=np.zeros_like(b)
  dL_dc=np.zeros_like(c)

  dL_dE=np.zeros_like(E) # for updating embedding vector

  dL_da_next=np.zeros_like(a[0]) # (64,1)

  for t in reversed(range(len(x_indx))):
    x_t=E[x_indx[t]]
    x_t=x_t.reshape(-1,1) # (9,1)

    y_true=np.zeros((vocab_size, 1))
    y_true[y_indx[t]]=1 # creating a one-hot vector for calculation

    dL_do=y_pred[t] - y_true # (17,1)

    # output gradients
    dL_dV+=dL_do@a[t].T # (17,64)
    dL_dc+=dL_do # (17,1)

    # input gradients
    dL_da=V.T @ dL_do+dL_da_next # (64,1)
    dL_ds=dL_da*derivative_tanh(s[t]) # (64,1)

    dL_db+=dL_ds # (64,1)
    dL_dW+=dL_ds@a[t-1].T # (64,64)
    dL_dU+=dL_ds@x_t.T # (64,9)

    dL_da_next=W.T @ dL_ds # (64,1)
    dL_dE[x_indx[t]] += (U.T @ dL_ds).flatten()

  return dL_dV, dL_dU, dL_dW, dL_db, dL_dc,dL_dE


In [ ]:
# dL_dV, dL_dU, dL_dW, dL_db, dL_dc,dL_dE  =BPTT(y_pred,U,V,W,b,c,E,x_indx,a,s,y_indx)


In [24]:
# let's train this model

epochs=100000
lr=0.01

for epoch in range(epochs):

    # training loop
    a_prev = np.zeros((H, 1))
    s,a,o,y_pred_train=forward_pass(W,U,V,b,c,a_prev,x_indx,E) # forward pass
    L_train=cross_entropy_loss(y_pred_train, y_indx)


    dL_dV, dL_dU, dL_dW, dL_db, dL_dc,dL_dE  =BPTT(y_pred_train,U,V,W,b,c,E,x_indx,a,s,y_indx)


    V-=lr*dL_dV
    c-=lr*dL_dc
    U-=lr*dL_dU
    W-=lr*dL_dW
    b-=lr*dL_db
    E-=lr*dL_dE

    # testing loop
    s, a, o, y_pred_test = forward_pass(W,U,V,b,c,a_prev,x_indx,E)
    L_test = cross_entropy_loss(y_pred=y_pred_test, y_true=y_indx)


    if (epoch%10000)==0:
        print(f'Epoch:{epoch}| Train Loss:{L_train}| Test Loss:{L_test}')

/tmp/ipython-input-101944338.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  loss-=float(np.log(y_pred[t][y_true[t]]+eps))


Epoch:0| Train Loss:0.0006757202829034494| Test Loss:0.0006757132276808453
Epoch:10000| Train Loss:0.0006117394539820782| Test Loss:0.0006117336523041241
Epoch:20000| Train Loss:0.0005586741784904275| Test Loss:0.0005586693251876425
Epoch:30000| Train Loss:0.0005139632368323312| Test Loss:0.000513959118087044
Epoch:40000| Train Loss:0.000475786725752939| Test Loss:0.00047578318736190845
Epoch:50000| Train Loss:0.0004428163179338919| Test Loss:0.00044281324591800153
Epoch:60000| Train Loss:0.00041406001857710844| Test Loss:0.00041405732688565945
Epoch:70000| Train Loss:0.00038876220483952405| Test Loss:0.0003887598273132234
Epoch:80000| Train Loss:0.00036633728230052546| Test Loss:0.00036633516724095526
Epoch:90000| Train Loss:0.00034632447748295494| Test Loss:0.00034632258391487276


In [25]:

def generate_text(seed_text, gen_length, E, W, U, V, b, c,
                  char_to_indx, indx_to_char, H):

    # seed_text : initial string
    # gen_length : number of characters to generate

    seed_indices = [char_to_indx[ch] for ch in seed_text]
    a_prev = np.zeros((H, 1))

    for idx in seed_indices[:-1]:
        x_t = E[idx].reshape(-1, 1)
        h_prev = np.tanh(W @ a_prev + U @ x_t + b)

    current_idx = seed_indices[-1]
    generated_text = seed_text

    for _ in range(gen_length):

        x_t = E[current_idx].reshape(-1, 1)

        h_t = np.tanh(W @ h_prev + U @ x_t + b)

        logits = V @ h_t + c

        y_pred = softmax(logits)

        # greedy sampling (other best methods also exists)
        next_idx = np.argmax(y_pred)

        # append character
        next_char = indx_to_char[next_idx]
        generated_text += next_char

        # update
        current_idx = next_idx
        h_prev = h_t

    return generated_text



In [26]:
seed = "diffusion works"
generated = generate_text(
    seed_text=seed,
    gen_length=40,
    E=E,
    W=W,
    U=U,
    V=V,
    b=b,
    c=c,
    char_to_indx=char_to_indx,
    indx_to_char=indx_to_char,
    H=H
)

print(generated)


diffusion worksion works better than autoregression wor
